In [ ]:
import os
import numpy as np
import pandas as pd
import anndata as ad
import subprocess
import matplotlib.pyplot as plt
import tempfile
from pathlib import Path
from scipy.stats import norm
import pyarrow.feather as feather
from concurrent.futures import ProcessPoolExecutor, as_completed

# Functions

## HETEROGENEITY UTILITIES

In [ ]:


def stouffer_meta(pvals, weights=None):
    pvals = np.asarray(pvals)
    valid = np.isfinite(pvals) & (pvals > 0) & (pvals < 1)
    if valid.sum() == 0:
        return np.nan

    z = norm.isf(pvals[valid])

    if weights is None:
        w = np.ones(valid.sum())
    else:
        w = np.asarray(weights)[valid]

    z_meta = np.sum(w * z) / np.sqrt(np.sum(w**2))
    return norm.sf(z_meta)


def i_squared(z_vals):
    z_vals = np.asarray(z_vals)
    valid = np.isfinite(z_vals)
    z = z_vals[valid]
    k = len(z)
    if k < 2:
        return np.nan

    Q = np.sum((z - z.mean())**2)
    return max(0, (Q - (k - 1)) / Q)


def tau_squared_dl(z_vals, weights=None):
    z_vals = np.asarray(z_vals)
    valid = np.isfinite(z_vals)
    z = z_vals[valid]
    k = len(z)

    if k < 2:
        return np.nan

    if weights is None:
        w = np.ones(k)
    else:
        w = np.asarray(weights)[valid]

    z_bar = np.sum(w * z) / np.sum(w)
    Q = np.sum(w * (z - z_bar)**2)

    denom = np.sum(w) - (np.sum(w**2) / np.sum(w))
    tau2 = max(0, (Q - (k - 1)) / denom)
    return tau2


def prediction_interval(z_vals, weights=None):
    z_vals = np.asarray(z_vals)
    valid = np.isfinite(z_vals)
    z = z_vals[valid]
    k = len(z)

    if k < 2:
        return np.nan, np.nan, np.nan

    if weights is None:
        w = np.ones(k)
    else:
        w = np.asarray(weights)[valid]

    z_meta = np.sum(w * z) / np.sqrt(np.sum(w**2))
    tau2 = tau_squared_dl(z, weights=w)
    se_pred = np.sqrt(tau2 + 1)

    PI_low = z_meta - 1.96 * se_pred
    PI_high = z_meta + 1.96 * se_pred

    return z_meta, PI_low, PI_high

In [ ]:
def to_dense_df(X, var_names, obs_names):
    """Return a dense pandas DataFrame cells×genes from AnnData.X."""
    if hasattr(X, "toarray"):
        arr = X.toarray()
    elif hasattr(X, "A"):
        arr = X.A
    else:
        arr = np.asarray(X)
    return pd.DataFrame(arr, index=obs_names, columns=var_names)

## R CALL WRAPPER USING TEMP FILES


In [ ]:
def run_tradeseq_single(expr_df, axis_vec, tradeseq_script, nknots=8, r_num_cores=4):
    """
    Run tradeSeq on a single replicate using temporary Feather files.
    expr_df: cells × genes (DataFrame)
    axis_vec: length n_cells (Series)
    """
    with tempfile.TemporaryDirectory() as tf:
        tf = Path(tf)
        expr_path = tf / "expr.feather"
        axis_path = tf / "axis.feather"
        out_csv = tf / "results.csv"

        # tradeSeq expects genes × cells
        expr_gc = expr_df.T.reset_index().rename(columns={"index": "gene"})
        axis_df = pd.DataFrame({"cell": axis_vec.index, "axis": axis_vec.values})

        feather.write_feather(expr_gc, expr_path)
        feather.write_feather(axis_df, axis_path)

        cmd = [
            "Rscript", tradeseq_script,
            "--expr", str(expr_path),
            "--axis", str(axis_path),
            "--out", str(out_csv),
            "--nknots", str(nknots),
            "--num_cores", str(r_num_cores),
        ]

        try:
            subprocess.run(cmd, check=True)
        except subprocess.CalledProcessError as e:
            print(f"[ERROR] tradeSeq failed: {e}")
            return None

        if not out_csv.exists():
            print("[WARN] tradeSeq produced no output for this replicate.")
            return None

        return pd.read_csv(out_csv)

## MAIN PIPELINE

In [ ]:
def run_tradeseq_pipeline(
    adata_path,
    tradeseq_script,
    celltype_field="cell_type",
    replicate_field="slide_id",
    axis_field="axis",
    layer=None, 
    min_cells=20,
    nknots=6,
    num_cores=4
):
    adata = ad.read_h5ad(adata_path)
    if axis_field not in adata.obs:
        raise ValueError(f"axis vector missing in adata.obs['{axis_field}']")

    results = {}   # {celltype: {replicate: df}}

    for ct in np.unique(adata.obs[celltype_field]):
        print(f"\n=== Cell type: {ct} ===")
        ad_ct = adata[adata.obs[celltype_field] == ct]
        reps = np.unique(ad_ct.obs[replicate_field])
        results[ct] = {}

        for rep in reps:
            print(f"  - replicate {rep}")
            ad_rep = ad_ct[(ad_ct.obs[replicate_field] == rep) & (~ad_ct.obs[axis_field].isna())]

            if ad_rep.n_obs < min_cells:
                print(f"    [SKIP] only {ad_rep.n_obs} cells (< {min_cells})")
                continue

            # convert sparse → dense when necessary
            X = ad_rep.X if layer is None else ad_rep.layers[layer]
            expr_df = to_dense_df(X, ad_rep.var_names, ad_rep.obs_names)

            axis_vec = ad_rep.obs[axis_field]
            # axis_vec = (axis_vec - axis_vec.min()) / (axis_vec.max() - axis_vec.min())

            res = run_tradeseq_single(expr_df, axis_vec, tradeseq_script,
                                      nknots=nknots, num_cores=num_cores)
            results[ct][rep] = res

    return results

## MP

In [ ]:
def _worker_tradeseq_job(job):
    """
    job: dict with keys:
      'celltype', 'rep', 'expr_df', 'axis_vec',
      'tradeseq_script', 'nknots', 'r_num_cores'
    """
    ct = job["celltype"]
    rep = job["rep"]
    expr_df = job["expr_df"]
    axis_vec = job["axis_vec"]
    tradeseq_script = job["tradeseq_script"]
    nknots = job["nknots"]
    r_num_cores = job["r_num_cores"]

    print(f"[worker] Running tradeSeq for {ct} / {rep} "
          f"({expr_df.shape[0]} cells × {expr_df.shape[1]} genes)")

    res = run_tradeseq_single(expr_df, axis_vec, tradeseq_script,
                              nknots=nknots, r_num_cores=r_num_cores)
    return ct, rep, res

In [ ]:
def run_tradeseq_pipeline_mp(
    adata_path,
    tradeseq_script,
    celltype_field="cell_type",
    replicate_field="slide_id",
    axis_field="axis",
    layer=None, 
    min_cells=20,
    nknots=8,
    r_num_cores=4,
    n_workers=4,
):
    """
    Parallel tradeSeq runner.
    - adata_path: .h5ad
    - tradeseq_script: path to run_tradeseq_auto.R
    - nknots: spline knots for tradeSeq
    - r_num_cores: cores per R call
    - n_workers: Python processes (jobs in parallel)
    """
    adata = ad.read_h5ad(adata_path)
    if axis_field not in adata.obs:
        raise ValueError(f"axis vector missing in adata.obs['{axis_field}']")

    jobs = []
    celltypes = np.unique(adata.obs[celltype_field])

    for ct in celltypes:
        ad_ct = adata[adata.obs[celltype_field] == ct]
        reps = np.unique(ad_ct.obs[replicate_field])

        for rep in reps:
            ad_rep = ad_ct[(ad_ct.obs[replicate_field] == rep) & (~ad_ct.obs[axis_field].isna())]
            if ad_rep.n_obs < min_cells:
                continue

            # convert sparse → dense when necessary
            X = ad_rep.X if layer is None else ad_rep.layers[layer]
            expr_df = to_dense_df(X, ad_rep.var_names, ad_rep.obs_names)

            axis_vec = ad_rep.obs[axis_field]
            # axis_vec = (axis_vec - axis_vec.min()) / (axis_vec.max() - axis_vec.min())
            
            jobs.append({
                "celltype": ct,
                "rep": rep,
                "expr_df": expr_df,
                "axis_vec": axis_vec,
                "tradeseq_script": tradeseq_script,
                "nknots": nknots,
                "r_num_cores": r_num_cores,
            })

    # Run jobs in parallel
    results = {ct: {} for ct in celltypes}
    
    with ProcessPoolExecutor(max_workers=n_workers) as ex:
        future_to_job = {ex.submit(_worker_tradeseq_job, job): job for job in jobs}

        for fut in as_completed(future_to_job):
            job = future_to_job[fut]
            ct = job["celltype"]
            rep = job["rep"]
            try:
                ct_out, rep_out, res_df = fut.result()
                results[ct_out][rep_out] = res_df
            except Exception as e:
                print(f"[ERROR] Job {ct}/{rep} failed in worker: {e}")
                results[ct][rep] = None

    return results

## META-ANALYSIS

In [ ]:
def meta_combine_results(results_dict):
    meta_out = {}

    for ct, rep_dict in results_dict.items():
        print(f"\n=== Meta-analysis: {ct} ===")
        dfs = [df for df in rep_dict.values() if df is not None]
        if len(dfs) == 0:
            continue

        all_genes = set().union(*[set(df["gene"]) for df in dfs])
        rows = []

        for gene in all_genes:
            assoc_pvals = []
            end_pvals = []

            for rep, df in rep_dict.items():
                if df is None:
                    continue
                row = df[df["gene"] == gene]
                if len(row) == 1:
                    assoc_pvals.append(row["association_pvalue"].values[0])
                    end_pvals.append(row["end_test_pvalue"].values[0])

            if len(assoc_pvals) == 0:
                continue

            assoc_z = norm.isf(assoc_pvals)
            end_z = norm.isf(end_pvals)

            # Meta p-values
            assoc_meta_p = stouffer_meta(assoc_pvals)
            end_meta_p = stouffer_meta(end_pvals)

            # Relative heterogeneity
            assoc_I2 = i_squared(assoc_z)
            end_I2 = i_squared(end_z)

            # Absolute heterogeneity
            assoc_tau2 = tau_squared_dl(assoc_z)
            end_tau2 = tau_squared_dl(end_z)

            assoc_tau = np.sqrt(assoc_tau2) if np.isfinite(assoc_tau2) else np.nan
            end_tau = np.sqrt(end_tau2) if np.isfinite(end_tau2) else np.nan

            # Prediction intervals
            assoc_zmeta, assoc_PI_low, assoc_PI_high = prediction_interval(assoc_z)
            end_zmeta, end_PI_low, end_PI_high = prediction_interval(end_z)

            rows.append([
                gene,
                assoc_meta_p, assoc_I2, assoc_tau2, assoc_tau,
                assoc_zmeta, assoc_PI_low, assoc_PI_high,
                end_meta_p, end_I2, end_tau2, end_tau,
                end_zmeta, end_PI_low, end_PI_high
            ])

        df_meta = pd.DataFrame(rows, columns=[
            "gene",
            "assoc_meta_p", "assoc_I2", "assoc_tau2", "assoc_tau",
            "assoc_zmeta", "assoc_PI_low", "assoc_PI_high",
            "end_meta_p", "end_I2", "end_tau2", "end_tau",
            "end_zmeta", "end_PI_low", "end_PI_high"
        ])

        # FDR
        df_meta["assoc_fdr"] = df_meta["assoc_meta_p"].rank() / len(df_meta)
        df_meta["end_fdr"] = df_meta["end_meta_p"].rank() / len(df_meta)

        meta_out[ct] = df_meta

    return meta_out

## Plotting Functions

In [ ]:
def plot_mean_vs_meta(df_meta, rep_dict, ct, outdir):
    outdir = Path(outdir); outdir.mkdir(exist_ok=True)

    rep_dfs = {rep: df.set_index("gene")
               for rep, df in rep_dict.items() if df is not None}

    rep_mat = pd.concat([df["association_pvalue"] for df in rep_dfs.values()], axis=1)
    rep_mat.columns = list(rep_dfs.keys())
    rep_mean = rep_mat.mean(axis=1)

    merged = df_meta.set_index("gene").join(rep_mean.rename("rep_mean_p")).reset_index()
    X = -np.log10(merged["rep_mean_p"]+1e-12)
    Y = -np.log10(merged["assoc_meta_p"]+1e-12)
    
    fig, ax = plt.subplots(figsize=(6,6))
    ax.scatter(X, Y, s=10, alpha=0.5)
    ax.plot([0,int(max(X))+1],[0,int(max(X))+1],"--",color="red")
    ax.set_xlabel("-log10(mean replicate p)")
    ax.set_ylabel("-log10(meta p)")
    ax.set_title(f"{ct}: mean vs meta")
    fig.tight_layout()
    fig.savefig(outdir / f"{ct}_mean_vs_meta.png", dpi=200)
    plt.close(fig)


def plot_rep_heatmap(df_meta, rep_dict, ct, outdir, top_n=50):
    outdir = Path(outdir); outdir.mkdir(exist_ok=True)

    top_genes = df_meta.sort_values("assoc_meta_p").head(top_n)["gene"]
    rep_dfs = {rep: df.set_index("gene")
               for rep, df in rep_dict.items() if df is not None}
    rep_mat = pd.concat([df.loc[top_genes]["association_pvalue"]
                         for df in rep_dfs.values()], axis=1)
    rep_mat.columns = list(rep_dfs.keys())

    data = -np.log10(rep_mat.values + 1e-12)

    fig, ax = plt.subplots(figsize=(10,8))
    im = ax.imshow(data, aspect='auto', cmap='viridis')
    ax.set_yticks(range(len(top_genes)))
    ax.set_yticklabels(top_genes)
    ax.set_xticks(range(len(rep_dfs)))
    ax.set_xticklabels(rep_dfs.keys(), rotation=90)
    fig.colorbar(im, ax=ax, label="-log10(p)")
    ax.set_title(f"{ct}: replicate heatmap")
    fig.tight_layout()
    fig.savefig(outdir / f"{ct}_heatmap.png", dpi=200)
    plt.close(fig)


def plot_meta_vs_I2(df_meta, ct, outdir):
    outdir = Path(outdir); outdir.mkdir(exist_ok=True)

    fig, ax = plt.subplots(figsize=(7,6))
    ax.scatter(df_meta["assoc_I2"], 
               -np.log10(df_meta["assoc_meta_p"]+1e-12),
               s=10, alpha=0.5)
    ax.set_xlabel("I²")
    ax.set_ylabel("-log10(meta p)")
    ax.set_title(f"{ct}: meta vs I²")
    fig.tight_layout()
    fig.savefig(outdir / f"{ct}_meta_vs_I2.png", dpi=200)
    plt.close(fig)


def plot_meta_vs_tau(df_meta, ct, outdir):
    outdir = Path(outdir); outdir.mkdir(exist_ok=True)

    fig, ax = plt.subplots(figsize=(7,6))
    ax.scatter(df_meta["assoc_tau"],
               -np.log10(df_meta["assoc_meta_p"]+1e-12),
               s=10, alpha=0.5)
    ax.set_xlabel("tau (absolute heterogeneity)")
    ax.set_ylabel("-log10(meta p)")
    ax.set_title(f"{ct}: meta vs tau")
    fig.tight_layout()
    fig.savefig(outdir / f"{ct}_meta_vs_tau.png", dpi=200)
    plt.close(fig)


def plot_PIwidth_vs_meta(df_meta, ct, outdir):
    outdir = Path(outdir); outdir.mkdir(exist_ok=True)
    pi_width = df_meta["assoc_PI_high"] - df_meta["assoc_PI_low"]

    fig, ax = plt.subplots(figsize=(7,6))
    ax.scatter(pi_width,
               -np.log10(df_meta["assoc_meta_p"]+1e-12),
               s=10, alpha=0.5)
    ax.set_xlabel("PI width")
    ax.set_ylabel("-log10(meta p)")
    ax.set_title(f"{ct}: PI width vs meta")
    fig.tight_layout()
    fig.savefig(outdir / f"{ct}_PIwidth_vs_meta.png", dpi=200)
    plt.close(fig)


def generate_all_plots(df_meta, results_dict, ct, outdir="plots"):
    plot_mean_vs_meta(df_meta, results_dict, ct, outdir)
    plot_rep_heatmap(df_meta, results_dict, ct, outdir)
    plot_meta_vs_I2(df_meta, ct, outdir)
    plot_meta_vs_tau(df_meta, ct, outdir)
    plot_PIwidth_vs_meta(df_meta, ct, outdir)

# Run

In [ ]:
# parameters
AD_PATH = "/home/x-aklein2/projects/aklein/BICAN/BG/data/BICAN_BG_CPS.h5ad"
R_SCRIPT = "/home/x-aklein2/projects/aklein/BICAN/BG/spatial_analysis/scripts/tradeseq.R"
OUTDIR = "/home/x-aklein2/projects/aklein/BICAN/BG/data/CPS/tradeseq_dsid"
image_path = "/home/x-aklein2/projects/aklein/BICAN/BG/images/CPS/tradeseq_dsid"
REP_KEY = "dataset_id"
CELLTYPE_KEY = "Subclass"
AXIS = "MS_NORM"

layer="counts"
min_cells = 50
num_cores = 4
nknots = 6

COORDS_MODE = "axis_only"
XY_KEYS = ("CENTER_X", "CENTER_Y")

In [ ]:
adata = ad.read_h5ad(AD_PATH)

In [ ]:
if AXIS not in adata.obs:
    raise ValueError(f"axis vector missing in adata.obs['{AXIS}']")

In [ ]:
# results = {}   # {celltype: {replicate: df}}
# for ct in np.unique(adata.obs[CELLTYPE_KEY])[1:]:
#     print(f"\n=== Cell type: {ct} ===")
#     ad_ct = adata[adata.obs[CELLTYPE_KEY] == ct]
#     reps = np.unique(ad_ct.obs[REP_KEY])
#     results[ct] = {}

#     for rep in reps:
#         print(f"  - replicate {rep}")
#         ad_rep = ad_ct[(ad_ct.obs[REP_KEY] == rep) & (~ad_ct.obs[AXIS].isna())]
#         if ad_rep.n_obs < min_cells:
#             print(f"    [SKIP] only {ad_rep.n_obs} cells (< {min_cells})")
#             continue

#         # convert sparse → dense when necessary
#         # X = ad_rep.X.toarray() if hasattr(ad_rep.X, "A") else ad_rep.X
#         # expr_df = pd.DataFrame(X, index=ad_rep.obs_names, columns=ad_rep.var_names)
#         X = ad_rep.X if layer is None else ad_rep.layers[layer]
#         expr_df = to_dense_df(X, ad_rep.var_names, ad_rep.obs_names)

#         axis_vec = ad_rep.obs[AXIS]
#         axis_vec = (axis_vec - axis_vec.min()) / (axis_vec.max() - axis_vec.min())

#         res = run_tradeseq_single(expr_df, axis_vec, R_SCRIPT,
#                                     nknots=nknots, num_cores=num_cores)
#         results[ct][rep] = res
        
#         # break
    
#     break

In [ ]:
# Tune these based on SLURM allocation:
N_WORKERS = 2        # Python processes (jobs in parallel)
R_NUM_CORES = 6      # cores per R tradeSeq call
NKNOTS = 8

results = run_tradeseq_pipeline_mp(
    adata_path=AD_PATH,
    tradeseq_script=R_SCRIPT,
    celltype_field=CELLTYPE_KEY,
    replicate_field=REP_KEY,
    axis_field=AXIS,
    layer=layer,
    nknots=NKNOTS,
    r_num_cores=R_NUM_CORES,
    n_workers=N_WORKERS,
    min_cells=min_cells
)

In [ ]:
# results = run_tradeseq_pipeline(
#     AD_PATH,
#     R_SCRIPT,
#     celltype_field=CELLTYPE_KEY,
#     replicate_field=REP_KEY,
#     axis_field=AXIS,
#     min_cells=min_cells,
#     layer=layer,
#     nknots=nknots,
#     num_cores=num_cores
# )

In [ ]:
meta = meta_combine_results(results)

for ct, df_meta in meta.items():
    # outdir = f"tradeseq_plots/{ct}"
    outdir="/home/x-aklein2/projects/aklein/BICAN/BG/images/CPS/sparkx/tt"
    generate_all_plots(df_meta, results[ct], ct, outdir)
    # df_meta.to_csv(f"tradeseq_meta_{ct}.csv", index=False)

In [ ]:
meta

# OLD

In [ ]:
import anndata as ad
import pandas as pd
import numpy as np
import subprocess
import os
from pathlib import Path
import pyarrow.feather as feather
from scipy.stats import norm
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#############################
# I/O HELPERS
#############################

def write_arrow_input(expr_df, axis_vec, base_path: Path):
    """
    Writes expression + axis to Feather (Arrow) format.
    expr_df: DataFrame cells × genes
    axis_vec: pandas Series indexed by cells
    """
    expr_path = base_path / "expr.feather"
    axis_path = base_path / "axis.feather"

    # tradeSeq expects genes × cells → transpose
    expr_gc = expr_df.T.reset_index().rename(columns={"index": "gene"})
    axis_df = pd.DataFrame({"axis": axis_vec.values}, index=axis_vec.index).reset_index().rename(columns={"index": "cell"})

    feather.write_feather(expr_gc, expr_path)
    feather.write_feather(axis_df, axis_path)

    return expr_path, axis_path

In [ ]:
#############################
# META-ANALYSIS UTILITIES
#############################

def stouffer_meta(pvals, weights=None):
    pvals = np.asarray(pvals)
    valid = np.isfinite(pvals) & (pvals > 0) & (pvals < 1)
    if valid.sum() == 0:
        return np.nan

    z = norm.isf(pvals[valid])
    if weights is None:
        weights = np.ones(valid.sum())
    else:
        weights = np.asarray(weights)[valid]

    z_meta = np.sum(weights * z) / np.sqrt(np.sum(weights**2))
    return norm.sf(z_meta)


def i_squared(z_vals):
    """
    Compute heterogeneity statistic I² across replicates.
    z_vals: array of replicate Z-scores
    """
    z_vals = np.asarray(z_vals)
    valid = np.isfinite(z_vals)
    z_vals = z_vals[valid]

    if len(z_vals) < 2:
        return np.nan

    Q = np.sum((z_vals - z_vals.mean())**2)
    df = len(z_vals) - 1
    return max(0, (Q - df) / Q)

def tau_squared_dl(z_vals, weights=None):
    """
    DerSimonian–Laird tau^2 estimator for absolute heterogeneity.
    """
    z_vals = np.asarray(z_vals)
    valid = np.isfinite(z_vals)
    z = z_vals[valid]

    k = len(z)
    if k < 2:
        return np.nan

    if weights is None:
        w = np.ones(k)
    else:
        w = np.asarray(weights)[valid]

    # Weighted mean
    z_bar = np.sum(w * z) / np.sum(w)

    # Q statistic
    Q = np.sum(w * (z - z_bar) ** 2)

    # DL denominator
    denom = np.sum(w) - (np.sum(w**2) / np.sum(w))
    tau2 = max(0, (Q - (k - 1)) / denom)
    return tau2


def prediction_interval(z_vals, weights=None):
    """
    Compute random-effects prediction interval for Z-meta.
    """
    z_vals = np.asarray(z_vals)
    valid = np.isfinite(z_vals)
    z = z_vals[valid]

    if len(z) < 2:
        return np.nan, np.nan, np.nan

    # Stouffer Z-meta
    if weights is None:
        w = np.ones(len(z))
    else:
        w = np.asarray(weights)[valid]
    z_meta = np.sum(w * z) / np.sqrt(np.sum(w**2))

    # Tau^2
    tau2 = tau_squared_dl(z, weights=w)

    # Prediction interval
    se_pred = np.sqrt(tau2 + 1)  # sampling variance ~1 for Z
    PI_low = z_meta - 1.96 * se_pred
    PI_high = z_meta + 1.96 * se_pred

    return z_meta, PI_low, PI_high


In [ ]:
#############################
# META-COMBINE RESULTS
#############################

def meta_combine_results(results_dict):
    """
    results_dict: {celltype: {replicate: df}}
    df contains: gene, association_pvalue, end_test_pvalue
    """
    meta_out = {}

    for ct, rep_dict in results_dict.items():
        print(f"\n=== Meta-analyzing cell type: {ct} ===")
        dfs = [df for df in rep_dict.values() if df is not None]
        if len(dfs) == 0:
            continue

        all_genes = set().union(*[set(df["gene"]) for df in dfs])
        df_meta_rows = []

        for gene in all_genes:
            assoc_pvals = []
            end_pvals = []

            for rep, df in rep_dict.items():
                if df is None:
                    continue
                sub = df[df["gene"] == gene]
                if len(sub) == 1:
                    assoc_pvals.append(sub["association_pvalue"].values[0])
                    end_pvals.append(sub["end_test_pvalue"].values[0])

            assoc_pvals = np.asarray(assoc_pvals)
            end_pvals = np.asarray(end_pvals)

            # Skip genes missing everywhere
            if assoc_pvals.size == 0:
                continue

            # Convert to Z
            assoc_z = norm.isf(assoc_pvals)
            end_z = norm.isf(end_pvals)

            # Meta p-values
            assoc_meta_p = stouffer_meta(assoc_pvals)
            end_meta_p = stouffer_meta(end_pvals)

            # Relative heterogeneity
            assoc_I2 = i_squared(assoc_z)
            end_I2 = i_squared(end_z)

            # Absolute heterogeneity
            assoc_tau2 = tau_squared_dl(assoc_z)
            end_tau2 = tau_squared_dl(end_z)

            assoc_tau = np.sqrt(assoc_tau2) if np.isfinite(assoc_tau2) else np.nan
            end_tau = np.sqrt(end_tau2) if np.isfinite(end_tau2) else np.nan

            # Prediction intervals
            assoc_zmeta, assoc_PI_low, assoc_PI_high = prediction_interval(assoc_z)
            end_zmeta, end_PI_low, end_PI_high = prediction_interval(end_z)

            df_meta_rows.append([
                gene,
                assoc_meta_p,
                assoc_I2,
                assoc_tau2,
                assoc_tau,
                assoc_zmeta,
                assoc_PI_low,
                assoc_PI_high,
                end_meta_p,
                end_I2,
                end_tau2,
                end_tau,
                end_zmeta,
                end_PI_low,
                end_PI_high
            ])

        df_meta = pd.DataFrame(df_meta_rows,
                         columns=[
                             "gene",
                             "assoc_meta_p", "assoc_I2", "assoc_tau2", "assoc_tau",
                             "assoc_zmeta", "assoc_PI_low", "assoc_PI_high",
                             "end_meta_p", "end_I2", "end_tau2", "end_tau",
                             "end_zmeta", "end_PI_low", "end_PI_high"
                         ])

        # FDR correction
        df_meta["assoc_fdr"] = (df_meta["assoc_meta_p"].rank() / len(df_meta))
        df_meta["end_fdr"] = (df_meta["end_meta_p"].rank() / len(df_meta))

        meta_out[ct] = df_meta

    return meta_out


In [ ]:
#############################
# RUN TRADSEQ PER REPLICATE
#############################

def run_tradeseq_single(expr_df, axis_vec, out_dir: Path, tradeseq_script, nknots=6, num_cores=1):
    """
    Run tradeSeq on a single subset (cell type × replicate).
    expr_df: cells × genes
    axis_vec: length = n_cells
    """

    out_dir.mkdir(parents=True, exist_ok=True)
    expr_path, axis_path = write_arrow_input(expr_df, axis_vec, out_dir)

    out_csv = out_dir / "tradeseq_results.csv"

    cmd = [
        "Rscript", tradeseq_script,
        "--expr", str(expr_path),
        "--axis", str(axis_path),
        "--out", str(out_csv),
        "--nknots", str(nknots),
        "--num_cores", str(num_cores)
    ]

    try:
        subprocess.run(cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"tradeSeq failed: {e}")
        return None

    if not out_csv.exists():
        return None

    return pd.read_csv(out_csv)

In [ ]:
#############################
# MAIN RUNNER
#############################

def run_tradeseq_pipeline(
    adata_path,
    tradeseq_script,
    celltype_field="cell_type",
    replicate_field="slide_id",
    output_dir="tradeseq_output",
    nknots=6,
    num_cores=4
):

    adata = ad.read_h5ad(adata_path)
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)

    # Store results here: {celltype: {replicate: df}}
    results = {}

    celltypes = np.unique(adata.obs[celltype_field])

    for ct in celltypes:
        print(f"\n=== Processing cell type: {ct} ===")
        ad_ct = adata[adata.obs[celltype_field] == ct]

        reps = np.unique(ad_ct.obs[replicate_field])
        results[ct] = {}

        for rep in reps:
            print(f"  - replicate: {rep}")

            ad_rep = ad_ct[ad_ct.obs[replicate_field] == rep]
            expr_df = pd.DataFrame(ad_rep.X.A if hasattr(ad_rep.X, "A") else ad_rep.X,
                                   index=ad_rep.obs_names,
                                   columns=ad_rep.var_names)

            # Ensure axis exists
            if "axis" not in ad_rep.obs:
                raise ValueError("AnnData must contain adata.obs['axis'] for spatial axis.")

            axis_vec = ad_rep.obs["axis"]

            rep_out = output_dir / f"{ct}__{rep}"
            rep_res = run_tradeseq_single(
                expr_df,
                axis_vec,
                rep_out,
                tradeseq_script,
                nknots=nknots,
                num_cores=num_cores,
            )

            results[ct][rep] = rep_res

    return results

In [ ]:
results = run_tradeseq_pipeline(
    adata_path="your_spatial_data.h5ad",
    tradeseq_script="run_tradeseq_auto.R",
    celltype_field="cell_type",
    replicate_field="slide_id",
    output_dir="tradeseq_runs",
    nknots=6,
    num_cores=8
)

meta = meta_combine_results(results)

In [ ]:
for ct, df in meta.items():
    # plot_top_genes(df, ct, outdir="tradeseq_summary_plots")
    df.to_csv(f"tradeseq_meta_{ct}.csv", index=False)


## Plots

In [ ]:
def plot_top_genes(df_meta, ct, outdir, n=20):
    outdir = Path(outdir)
    outdir.mkdir(exist_ok=True)

    df_plot = df_meta.sort_values("association_meta_p").head(n)

    plt.figure(figsize=(6,8))
    sns.barplot(
        data=df_plot,
        x=-np.log10(df_plot["association_meta_p"] + 1e-12),
        y="gene",
        hue="association_I2",
        dodge=False
    )
    plt.title(f"{ct}: Top {n} axis-associated genes")
    plt.xlabel("-log10(meta p)")
    plt.ylabel("")
    plt.tight_layout()
    plt.savefig(outdir / f"{ct}_top_genes.png", dpi=200)
    plt.close()


In [ ]:
def plot_pval_heatmap(df_meta, results_dict, ct, outdir, top_n=50):
    outdir = Path(outdir); outdir.mkdir(exist_ok=True)

    top_genes = df_meta.sort_values("assoc_meta_p").head(top_n)["gene"]

    rep_dfs = {rep: df.set_index("gene") for rep, df in results_dict.items() if df is not None}
    rep_mat = pd.concat([df.loc[top_genes]["association_pvalue"] for df in rep_dfs.values()], axis=1)
    rep_mat.columns = list(rep_dfs.keys())

    data = -np.log10(rep_mat.values + 1e-12)

    fig, ax = plt.subplots(figsize=(10,8))
    im = ax.imshow(data, aspect='auto', cmap="viridis")
    ax.set_yticks(range(len(top_genes)))
    ax.set_yticklabels(top_genes)
    ax.set_xticks(range(len(rep_dfs)))
    ax.set_xticklabels(list(rep_dfs.keys()), rotation=90)
    fig.colorbar(im, ax=ax, label="-log10(p)")
    ax.set_title(f"{ct}: replicate p-values (top {top_n})")
    fig.tight_layout()
    fig.savefig(outdir / f"{ct}_replicate_heatmap.png", dpi=200)
    plt.close(fig)


In [ ]:
def plot_variance_vs_meta(df_meta, results_dict, ct, outdir):
    outdir = Path(outdir); outdir.mkdir(exist_ok=True)

    rep_dfs = {rep: df.set_index("gene") for rep, df in results_dict.items() if df is not None}
    rep_mat = pd.concat([df["association_pvalue"] for df in rep_dfs.values()], axis=1)

    rep_var = rep_mat.var(axis=1)

    merged = df_meta.set_index("gene").join(rep_var.rename("rep_var")).reset_index()

    fig, ax = plt.subplots(figsize=(7,6))
    ax.scatter(merged["rep_var"], -np.log10(merged["assoc_meta_p"]+1e-12),
               s=10, alpha=0.4)
    ax.set_xlabel("Replicate p-value variance")
    ax.set_ylabel("-log10(meta p)")
    ax.set_title(f"{ct}: variance vs meta p")
    fig.tight_layout()
    fig.savefig(outdir / f"{ct}_variance_vs_meta.png", dpi=200)
    plt.close(fig)


In [ ]:
def plot_mean_vs_meta(df_meta, results_dict, ct, outdir):
    outdir = Path(outdir)
    outdir.mkdir(exist_ok=True)

    # Build replicate p-value matrix
    rep_dfs = {rep: df.set_index("gene") for rep, df in results_dict.items() if df is not None}
    rep_cols = []
    for rep, df in rep_dfs.items():
        rep_cols.append(df["association_pvalue"])
    rep_mat = pd.concat(rep_cols, axis=1)
    rep_mat.columns = list(rep_dfs.keys())

    rep_mean = rep_mat.mean(axis=1)

    merged = df_meta.set_index("gene").join(rep_mean.rename("rep_mean_p")).reset_index()

    fig, ax = plt.subplots(figsize=(6,6))
    ax.scatter(-np.log10(merged["rep_mean_p"]+1e-12),
               -np.log10(merged["assoc_meta_p"]+1e-12),
               s=10, alpha=0.5)
    ax.plot([0,20], [0,20], linestyle="--", color="red")
    ax.set_xlabel("-log10(mean replicate p)")
    ax.set_ylabel("-log10(meta p)")
    ax.set_title(f"{ct}: mean replicate vs meta")
    fig.tight_layout()
    fig.savefig(outdir / f"{ct}_mean_vs_meta.png", dpi=200)
    plt.close(fig)


In [ ]:
def plot_meta_vs_I2(df_meta, ct, outdir):
    outdir = Path(outdir); outdir.mkdir(exist_ok=True)

    fig, ax = plt.subplots(figsize=(7,6))
    ax.scatter(df_meta["assoc_I2"], -np.log10(df_meta["assoc_meta_p"]+1e-12),
               s=10, alpha=0.4)
    ax.set_xlabel("I² (relative heterogeneity)")
    ax.set_ylabel("-log10(meta p)")
    ax.set_title(f"{ct}: Meta significance vs I²")
    fig.tight_layout()
    fig.savefig(outdir / f"{ct}_meta_vs_I2.png", dpi=200)
    plt.close(fig)


In [ ]:
def plot_meta_vs_tau(df_meta, ct, outdir):
    outdir = Path(outdir); outdir.mkdir(exist_ok=True)

    fig, ax = plt.subplots(figsize=(7,6))
    ax.scatter(df_meta["assoc_tau"], -np.log10(df_meta["assoc_meta_p"]+1e-12),
               s=10, alpha=0.4)
    ax.set_xlabel("tau (absolute heterogeneity)")
    ax.set_ylabel("-log10(meta p)")
    ax.set_title(f"{ct}: Meta significance vs tau")
    fig.tight_layout()
    fig.savefig(outdir / f"{ct}_meta_vs_tau.png", dpi=200)
    plt.close(fig)


In [ ]:
def plot_PIwidth_vs_meta(df_meta, ct, outdir):
    outdir = Path(outdir); outdir.mkdir(exist_ok=True)

    pi_width = df_meta["assoc_PI_high"] - df_meta["assoc_PI_low"]

    fig, ax = plt.subplots(figsize=(7,6))
    ax.scatter(pi_width, -np.log10(df_meta["assoc_meta_p"]+1e-12),
               s=10, alpha=0.4)
    ax.set_xlabel("Prediction interval width")
    ax.set_ylabel("-log10(meta p)")
    ax.set_title(f"{ct}: PI width vs meta-p")
    fig.tight_layout()
    fig.savefig(outdir / f"{ct}_PIwidth_vs_meta.png", dpi=200)
    plt.close(fig)


In [ ]:
def generate_all_plots(df_meta, results_dict, ct, outdir="plots"):
    plot_mean_vs_meta(df_meta, results_dict, ct, outdir)
    plot_pval_heatmap(df_meta, results_dict, ct, outdir)
    plot_variance_vs_meta(df_meta, results_dict, ct, outdir)
    plot_meta_vs_I2(df_meta, ct, outdir)
    plot_meta_vs_tau(df_meta, ct, outdir)
    plot_PIwidth_vs_meta(df_meta, ct, outdir)
